<a href="https://colab.research.google.com/github/DorZarka/CV_Final_Project/blob/master/ObjectMeasurment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math

In [ ]:
def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [ ]:
if __name__ == '__main__':
    cap = cv.VideoCapture(0)
    if not cap.isOpened():
        print("Camera detection failed")
    else:
        while True:
            # read configuration
            ret, frame = cap.read()

            # find contours
            gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            smoothed = cv.GaussianBlur(gray_frame, (13, 13), 0)
            ret, thresh = cv.threshold(smoothed, 80, 255, 0)
            edged = cv.Canny(thresh, 20, 60)
            contours, hierarchy = cv.findContours(edged, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

            # draw approximated contours
            for cnt in contours:
                epsilon = 0.005 * cv.arcLength(cnt, True)
                approx = cv.approxPolyDP(cnt, epsilon, True)
                # cv.drawContours(frame, [approx], 0, (0, 255, 0), 3)

            # draw rotated contours
                rect = cv.minAreaRect(approx)
                box = cv.boxPoints(rect)
                box = np.intp(box)
                # cv.drawContours(frame, [box], 0, (0, 0, 255), 2)

            # calculate size
                width = int(distance(box[0], box[1]))
                height = int(distance(box[0], box[3]))

            # get bounding box
                x, y, w, h = cv.boundingRect(cnt)

            # put text (write the object size)
                font = cv.FONT_HERSHEY_SIMPLEX
                org = (x, y + h + 20)
                fontScale = 0.5
                color = (255, 0, 0)
                thickness = 1
                frame = cv.putText(frame, "width: {} | height: {}".format(width, height), org, font,
                                   fontScale, color, thickness, cv.LINE_AA)

            # draw bounding box
                cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # show the result
            cv.imshow('frame', frame)

            # exit program (by pressing q on keyboard)
            if cv.waitKey(1) == ord('q'):
                break


    cap.release()
    cv.destroyAllWindows()

Camera detection failed
